In [1]:
import sys, os
%load_ext autoreload
%autoreload 2
import torch
import torch.nn.functional as F
from torch.autograd import grad
import torch.distributions as distrib
import torch.nn as nn
import torch.utils.data as td
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
plt.switch_backend('TKAgg') #('Qt5Agg')
from scipy.sparse import coo_matrix
#from envs import Discrete_Pos_Trap, Gym_Env, Print_Env
from torch.distributions.utils import lazy_property
import foundation as fd
from foundation.util import NS
#import ray
import sequential_ray as ray
from ray_rl_backend import *
from ray_rl_core import *
print('prep done')

/home/fleeb/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


using sequential
prep done


In [2]:
ray.init()

Sequential Ray initialized.


In [3]:
args = NS()
args.name = 'test'

args.log = True
args.save_root = 'results'

args.def_type = 'torch.FloatTensor'

args.env = 'cartpole'
#args.env = 'pendulum'
args.base_seed = 12

args.reg_coeff = 1e-6
args.time_order = 3
args.obs_order = 1
args.max_steps = 10
args.nonlinearity = 'elu'
args.hidden = [3]

args.delta = 0.01
args.gamma = 0.99

args.num_iter = 100
args.num_traj = 20
args.num_eval = 2

print('Name: {}'.format(args.name))

Name: test


In [4]:
now = time.strftime("%b-%d-%Y-%H%M%S")
args.save_dir = os.path.join(args.save_root, args.name, now)
tblogger = None
if args.log:
    util.create_dir(args.save_dir)
    tblogger = util.TBLogger(args.save_dir)  # Start tensorboard logger
    tblogger.scalar_summary('zzz-ignore', 0, 0)
    print('Logging')
    # Create logfile to save prints
    #logfile = open(os.path.join(args.save_dir, 'logfile.txt'), 'a+')
    #backup = sys.stdout
    #sys.stdout = util.Tee(sys.stdout, logfile)
env = util.get_env(args.env)
obs_dim = env.spec.obs_space.size
act_dim = env.spec.action_space.choices if args.env == 'cartpole' else env.spec.action_space.size
print('Save dir: {}'.format(args.save_dir))
print('Env: {} (obs={},act={})'.format(args.env, obs_dim, act_dim))

Logging
Save dir: results/test/May-22-2018-132418
Env: cartpole (obs=4,act=2)


In [5]:
model = fd.nets.MLP(input_dim=obs_dim, output_dim=act_dim, hidden_dims=args.hidden, nonlinearity=args.nonlinearity)
print(model)
#policy = fd.policies.Gaussian_Policy(model=model, def_type=args.def_type, init_log_std=0, min_log_std=-3)
policy = fd.policies.Categorical_Policy(model=model, def_type=args.def_type)
baseline = None
total_episodes = 0

MLP(
  (net): Sequential(
    (0): BasicLinear(
      (layer): Linear(in_features=4, out_features=3, bias=True)
      (nonlin): ELU(alpha=1.0, inplace)
    )
    (1): BasicLinear(
      (layer): Linear(in_features=3, out_features=2, bias=True)
    )
  )
)


In [6]:
seed = args.base_seed
score = None
stats = util.StatsMeter('score')
time_stats = util.StatsMeter('traj', 'baseline', 'train', 'eval')
best_perf = None
for itr in range(args.num_iter):
    start = time.time()
    
    # generate paths
    path_ids = [generate_path.remote(policy, env, baseline=baseline, discount=args.gamma, seed=i+seed, obs_order=args.obs_order, time_order=args.time_order) 
                     for i in range(args.num_traj)]
    seed += args.num_traj
    paths = collate(ray.get(path_ids))
    time_stats.update('traj', time.time() - start)
    start = time.time()
    
    # train baseline
    baseline, bsln_stats = ray.get(train_baseline.remote(baseline, paths, reg_coeff=args.reg_coeff, obs_order=args.obs_order, time_order=args.time_order))
    stats.join(bsln_stats, prefix='bsln-')
    time_stats.update('baseline', time.time() - start)
    start = time.time()
    
    # train policy
    policy, optim_stats = ray.get(npg_update.remote(policy, paths, def_type=args.def_type, reg_coeff=args.reg_coeff, step_size=args.delta, max_cg_iter=args.max_steps))
    stats.join(optim_stats, prefix='optim-')
    score = stats['optim-returns'].avg if score is None else score*0.9 + stats['optim-returns'].avg*0.1
    stats.update('score', score)
    total_episodes += args.num_traj
    time_stats.update('train', time.time() - start)
    start = time.time()
    
    # eval
    eval_stats = [evaluate_policy.remote(policy, env, seed=i+seed) for i in range(args.num_eval)]
    seed += args.num_eval
    for estat in eval_stats:
        stats.join(ray.get(estat), prefix='eval-')
    time_stats.update('eval', time.time() - start)
    
    if best_perf is None or stats['eval-reward'].avg >= best_perf:
        best_perf = stats['eval-reward'].avg
    print_stats(stats, time_stats, best_perf=best_perf, itr=itr, num_iter=args.num_iter, logger=tblogger, total_episodes=total_episodes, verbose=not args.log)
    #print(list(policy.parameters())[-1])
    # save model
    

/mnt/c/Users/anwan/OneDrive - UW Office 365/Khan/DeepRL/project/ray_rl_backend.py:172: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  b
/mnt/c/Users/anwan/OneDrive - UW Office 365/Khan/DeepRL/project/foundation/util/stats.py:178: RuntimeWarning: invalid value encountered in sqrt
  self.std = np.sqrt(self.var)
/mnt/c/Users/anwan/OneDrive - UW Office 365/Khan/DeepRL/project/foundation/util/stats.py:162: RuntimeWarning: invalid value encountered in sqrt
  self.std = np.sqrt(self.var)


[ 05-22-18 13:24:19 ] 1/100 (ep=20) : 13.290 9.500   9.5 
[ 05-22-18 13:24:19 ] 2/100 (ep=40) : 9.284 11.375  11.4 
[ 05-22-18 13:24:19 ] 3/100 (ep=60) : 8.063 18.344  18.3 
[ 05-22-18 13:24:19 ] 4/100 (ep=80) : 7.037 11.336  18.3 
[ 05-22-18 13:24:20 ] 5/100 (ep=100) : 6.163 9.834  18.3 
[ 05-22-18 13:24:20 ] 6/100 (ep=120) : 6.554 9.458  18.3 
[ 05-22-18 13:24:20 ] 7/100 (ep=140) : 6.369 9.365  18.3 
[ 05-22-18 13:24:20 ] 8/100 (ep=160) : 6.828 11.841  18.3 
[ 05-22-18 13:24:21 ] 9/100 (ep=180) : 6.219 10.210  18.3 
[ 05-22-18 13:24:21 ] 10/100 (ep=200) : 5.893 12.053  18.3 
[ 05-22-18 13:24:22 ] 11/100 (ep=220) : 6.316 18.763  18.8 
[ 05-22-18 13:24:22 ] 12/100 (ep=240) : 7.779 14.191  18.8 
[ 05-22-18 13:24:22 ] 13/100 (ep=260) : 8.858 18.798  18.8 
[ 05-22-18 13:24:22 ] 14/100 (ep=280) : 9.635 17.949  18.8 
[ 05-22-18 13:24:23 ] 15/100 (ep=300) : 9.229 16.737  18.8 
[ 05-22-18 13:24:23 ] 16/100 (ep=320) : 9.337 14.684  18.8 
[ 05-22-18 13:24:23 ] 17/100 (ep=340) : 8.822 17.671  18